## Calculations for PHREEQC parameters

The excel calculations are transcribed in the below cell. The default module dimensions and characteristics are parameterized from the BW30-400 commerical module from Dow Chemical. 

In [ ]:
#import libraries
import math


#unit conversions 
mm_over_m = 1E3 
mm_over_inch = 25.4
nm_over_mm = 1E6
grams_over_kilograms = 1E3
hours_over_day = 24
minutes_over_hour = 60
seconds_over_minute = 60
liters_over_cubic_meter = 1E3


#calculation constants
grams_over_liters_h2o = 997.07 # @25 degrees celcius
grams_over_moles_h2o = 18.015 
kinematic_flow_velocity = 9.33E-7    #square meters / second
simulated_time_over_computational_time = 9.29    


#simulation constants
timesteps_over_cell = 1


#parameterize the module 
print('RO module dimensions:')
module_unit = input('Will you simulate a BW30-400 module? ; \'y\' or \'n\'  ')
if module_unit == 'y':
    module_diameter =  201                   #mm
    permeate_tube_diameter =  29             #mm
    module_length =  1.016                   #m
    permeate_flow = 40                       #cubic meters / day
    max_feed_flow = 15.9                     #cubic meters / hour
    membrane_thickness = 250 / nm_over_mm    #nm
    feed_thickness = 0.7112                  #mm
    permeate_thickness = 0.3                 #mm
    polysulfonic_layer_thickness = 0.05      #mm 
    support_layer_thickness = 0.15           #mm
    repeated_membrane_thickness = 2 * membrane_thickness + feed_thickness + permeate_thickness + 2 * polysulfonic_layer_thickness + 2 * support_layer_thickness      #mm
    print('\nMembrane thickness:', repeated_membrane_thickness, '(mm)')

else:
    membrane_knowledge = input('Do you know the thickness of the repeated module unit? ; \'y\' or \'n\'  ')

    if membrane_knowledge == 'y':
        repeated_membrane_thickness = input('RO membrane thickness (mm) ; Default = 1.4117  ') or 1.4117

    else:   
        module_diameter = int(input('Module diameter (mm)  '))
        permeate_tube_diameter = int(input('Module permeate tube diameter (mm)  '))
        module_length = int(input('Module length (m)  ')) 
        permeate_flow = int(input('Permeate flow rate (m^3 / day)  ')) 
        max_feed_flow = int(input('Maximum feed flow rate (m^3 / hour)  '))
        membrane_thickness = int(input('Filtration membrane thickness (nm) ; Default = 250  ')) / nm_over_mm
        feed_thickness = int(input('Feed spacer thickness (mm) ; Default = 0.7112  ')) 
        permeate_thickness = int(input('Permeate spacer thickness (mm) ; Default = 0.3  '))
        polysulfonic_layer_thickness = int(input('Polysulfonic layer thickness (mm) ; Default = 0.05  '))
        support_layer_thickness = int(input('Support layer thickness (mm) ; Default = 0.15  ')) 
        repeated_membrane_thickness = 2 * membrane_thickness + feed_thickness + permeate_thickness + 2 * polysulfonic_layer_thickness + 2 * support_layer_thickness
        print('Membrane thickness:', repeated_membrane_thickness)
        

#parameterize simulation characteristics
print('\nParameterize the simulation:')
try:
    simulation_shifts = int(input('Quantity of simulation shifts ; Default = 24  '))
except:
    simulation_shifts = 24
try:
    cells_per_module = int(input('Quantity of discretized module cells ; Default = 12  '))
except:
    cells_per_module = 12
try:
    max_feed_flow = float(input('Maximum feed flow rate (m^3 / hour) ; Default = 15.9  ')) 
except:
    max_feed_flow = 15.9
try:
    quantity_of_modules = int(input('Quantity of in-series RO modules ; Default = 1  '))
except:
    quantity_of_modules = 1


#calculate module properties
module_cross_sectional_area = module_diameter**2 * math.pi / 4        #squared millimeters
permeate_tube_cross_sectional_area = permeate_tube_diameter**2 * math.pi / 4     #squared millimeters
filtering_layer_thicknes = (module_diameter - permeate_thickness) / 2         #millimeters
filtration_cross_sectional_area = module_cross_sectional_area - permeate_tube_cross_sectional_area        #squared millimeters
module_windings = filtering_layer_thicknes / filtration_cross_sectional_area        #complete revolutions
cell_length = module_length / cells_per_module     #meters


#feed layer calculations
feed_cross_sectional_area = (feed_thickness / repeated_membrane_thickness) * filtration_cross_sectional_area     #squared millimeters
#print('\nFeed cross-sectional area:', feed_cross_sectional_area)
feed_volume = feed_cross_sectional_area * module_length / mm_over_m**2   #cubic meters
#print('Feed volume:', feed_volume)
feed_mass = feed_volume * grams_over_liters_h2o * liters_over_cubic_meter / grams_over_kilograms    #kilograms, which assumes pure water for mass
#print('Feed mass', feed_mass)
feed_moles = feed_mass * grams_over_kilograms / grams_over_moles_h2o 
#print('Feed moles:', feed_moles)

#fluid flow calculations
feed_velocity = max_feed_flow / (feed_cross_sectional_area / mm_over_m**2) / minutes_over_hour / seconds_over_minute     #meters / second
reynolds_number = feed_velocity * module_length / kinematic_flow_velocity

#module cell calculations
feed_mass_cell = feed_mass / cells_per_module      #kg
feed_moles_cell = feed_moles / cells_per_module    #moles


#calculate simulation parameters that will populate the PHREEQC simulation 
maximal_timestep = cell_length / feed_velocity * timesteps_over_cell         #seconds, from the Courant condition
#print(maximal_timestep)
permeate_removal_per_cell = maximal_timestep * permeate_flow / (hours_over_day * minutes_over_hour * seconds_over_minute) * liters_over_cubic_meter * grams_over_liters_h2o / grams_over_moles_h2o / cells_per_module      #moles / cell
#print(permeate_removal_per_cell)    
    
    

#REACTION calculations 
print('''Permeate removal:\n 
        first approach: linear removal of moles\n
        second approach: linear change in concentration\n''')
permeate_approach = input('What approach of permeate removal will you simulate? < first > or < second >  ')
cfs = []
cell_moles = []
reaction_parameters = []
iteration = 0
previous_moles_removed = 0
cumulative_cf = 1 
for module in range(quantity_of_modules):
    if permeate_approach == 'first':
        if iteration == 0:
            initial_moles_cell = feed_moles_cell
        #print('\nInitial moles:\n', initial_moles_cell)
        print('''Permeate efficiency (PE):
        The PE embodies flux decreases through the module from fouling. 
        A value of 0 denotes complete fouling and the absence of permeate flux.
        A value of 1 denotes the absence of fouling and unabated permeate flux.'''))
        permeate_efficiency = float(input('''What is the permeate efficiency (PE) for this module?
        0 < x < 1  '''))
        print('''Presesure gradient: 
        A value of -10 denotes a 10-log reduction in pressure at the end of the module realtive to the start of the module.
        A value of 0 denotes a homogeneous pressure throughout the module.''') 
        osmotic_pressure = float(input('''What is the pressure gradient through this module? 
        -10 < x < 0  ''')
        initial_moles_removed = permeate_removal_per_cell * 2 / (1 + math.exp(osmotic_pressure))
        final_moles_removed = initial_moles_removed * math.exp(osmotic_pressure)
        removed_moles_slope = ((final_moles_removed - initial_moles_removed) / (cells_per_module)) * permeate_efficiency
        average_moles_removed = (final_moles_removed + initial_moles_removed) / 2
        #print('Average moles removed: ', average_moles_removed)
        #print('\nInitial moles removed:\n', initial_moles_removed)
        #print('\nFinal moles removed:\n',final_moles_removed)
        #print('\nCells per module:\n',cells_per_module)
        print('Removed moles slope:', removed_moles_slope)

        permeate_efficiency_power = 1.1
        for cell in range(cells_per_module):
            removed_moles_in_cell = (cell * removed_moles_slope + initial_moles_removed) * permeate_efficiency**permeate_efficiency_power
            reaction_parameters.append(removed_moles_in_cell)
            previous_moles_removed += removed_moles_in_cell 
            #print(previous_moles_removed)
            
        #print('Removed moles: ', previous_moles_removed)
        cf = initial_moles_cell / (initial_moles_cell - previous_moles_removed)
        #print(cf)
        cumulative_cf *= cf
        #print('Cumulative CF:', cumulative_cf)
        
        initial_moles_cell -= previous_moles_removed
        previous_moles_removed = 0
        iteration += 1
        
        
    if permeate_approach == 'second':
        initial_cf = 1
        final_cf = float(input("What is the final CF of your simulation? "))
        cf_slope = (final_cf - initial_cf) / cells_per_module
        #print(cf_slope)
        for cell in range(1,cells_per_module+1):
            cell_cf = cell * cf_slope + 1
            cfs.append(cell_cf)    

        #print(cfs)

        for cf in cfs:
            moles_to_be_removed = (feed_moles / cf) - feed_moles
            if iteration == 0:
                reaction_parameters.append(moles_to_be_removed)
            if iteration > 0:
                previous_moles_removed += reaction_parameters[iteration-1] 
                #print(reaction_parameters)
                #print('to_be:', moles_to_be_removed)
                #print('previous:', previous_moles_removed)
                reaction_parameter = moles_to_be_removed - previous_moles_removed
                reaction_parameters.append(reaction_parameter)

            iteration += 1
            
        cf = cfs[-1]
        #print(cf)
        cumulative_cf *= cf
        print('Cumulative CF: ', cumulative_cf)
    
#print(reaction_parameters)   
#print(len(reaction_parameters))
    
    
    
    
'''
Purpose:
    calculate values from user inquiries 
'''
def user_inquiries():
    
    #the calculations for the mass and Cf are inverse
    inquiry = input('Do you want to find the < mass > or < CF > of a solution?  ')
    if inquiry == 'CF':
        hypothesized_solution_mass = float(input('What is hypothsized solution mass? (kg)  '))
        concentration_factor_from_mass = (hypothesized_solution_mass * grams_over_kilograms / grams_over_moles_h2o) / ((hypothesized_solution_mass * grams_over_kilograms / grams_over_moles_h2o) - (permeate_removal_per_cell / cells_per_module))
        print(concentration_factor_from_mass)
        
    if inquiry == 'mass':
        hypothesized_solution_cf = float(input('What is the hypothesized solution CF?  '))
        try:
            mass_from_concentration_factor = (hypothesized_solution_cf * permeate_removal_per_cell) / (hypothesized_solution_cf - 1) * (grams_over_moles_h2o / grams_over_kilograms) 
        except:
            mass_from_concentration_factor = feed_mass_cell
        print(mass_from_concentration_factor)
'''
Purpose:
    simulation results and attributes are printed 
'''  
def printed_results():
    simulation_time = maximal_timestep * simulation_shifts
    computational_time = simulation_time / simulated_time_over_computational_time
    print('\nEstimated computational time:\n', computational_time, '(s)')

    

printed_results()

## iPHREEQpy simulation

Example 11 of iPHREEQC is emulated in PHREEQpy. 

In [ ]:
#import statements

from __future__ import print_function
import sys
import os
import timeit
import datetime
import phreeqpy.iphreeqc.phreeqc_com as phreeqc_mod

#===================================================================================
#example 11 from PHREEQC in the PHREEQpy examples

def make_general_conditions():
    global database_selection
    
    iphreeqc_path = input('''What is the path directory of your iPHREEQC software?
    e.g. C:\Program Files\USGS\IPhreeqcCOM 3.6.2-15100''')        #input a function which locates whether a file of previous defined paths exists. The absence of a path would trigger the input prompt for the path insertion. 

    existing_simulation_parameters = input('Do you have a file of general simulation parameters? < y > or < n >')
    if existing_simulation_parameters == 'y':
        simulation_parameter_path = input('''What is the path of the general simulation parameters?
        e.g. C:\Users\Cruella Deville\Documents\PHREEQpy\general simulation parameters''')
        
    else:    
        database_selection = input('''What database do you select? 
        < pitzer >, < phreeqc >, < Amm >, < ColdChem >, < core10 >, 
        < frezchem >, < iso >, < llnl >, < minteq >, <minteq.v4 >, 
        < sit >, < Tipping_Hurley >, < wateq4f >''') 
        database_path = os.path.join(iphreeqc_path, 'database', '%s.dat' %(database_selection))
        simulation_title = input('What is the title of your simulation?')   
        database_line = 'DATABASE %s %(database_path)'
        title_line = 'TITLE %s %(simulation_title)'
 

def make_solutions():
    """
    Specify initial conditions data blocks.

    Uniform initial conditions are assumed.
    """
    solution_block = []
    #create the solution line of the input file
    solution_number = input('''We will now create the SOLUTION block. To what cell(s) does this solution apply? 
    < # > or < #-# > ; Default = 1''') or 1
    solution_description = input('''What is the solution description?
    Default is none''')
    solution_line = 'SOLUTION %d %s' % (solution_number, solution_description)
    solution_block.append(solution_line)
    
    
    #create the temperature line of the input file
    solution_temperature = input('''What is the solution temperature? 
    Default = 25 (K)''')
    temperature_line = 'temp \t\t %d' %(solution_temperature)
    solution_block.append(temperature_line)
    
    
    #create the pH line of the input file
    solution_ph = input('What is the solution pH?')
    ph_charge_balance = input('''Is this pH charge balanced?
    < y > or < n >''')
    if ph_charge_balance == 'y':
        solution_ph += ' charge'
    ph_line = 'pH \t\t %s' %(solution_ph)
    solution_block.append(ph_line)
    
    
    #create the pe line of the input file
    solution_pe = input('''What is the solution pe (-log(electron activity))? 
    Default = 4.0''') or 4
    if solution_pe != 0:
        pe_charge_balance = input('''Is this pe charge balanced? 
        < y > or < n >''')
        if pe_charge_balance == 'y':
            pe_charge = ' charge'

        elif pe_charge_balance == 'n':
            pe_phase_balance = input('''Is the pe phase balanced? 
            < y > or < n >''')
            if pe_phase_balance == 'y':
                pe_phase = input('What is the balanced phase name?')
                pe_phase_index = input('''What is the phase saturation index?
                Default = 0''') or 0
                pe_charge = ' %s %s' %(pe_phase, pe_phase_index) 
    pe_line = 'pe \t\t ' + solution_pe + pe_charge
    solution_block.append(pe_line)
        
        
    #create the redox line of the input file      
    solution_redox = input('''Is a special pe calculated via a redox reaction? 
    < y > or < n >''')
    if solution_redox == 'y':
        redox_couple = input('''What is the redox couple?
        e.g. O(-2)/O(0) for oxygen with (-2) and (0) oxidation states''')
    redox_line = 'redox \t\t ' + redox_couple
    solution_block.append(redox_line)
            
        
    #create the units line of the input file        
    solution_units = input('''What are the concentration units of your solution? 
    numerator = < mg >, < ug >, < mmol >, < umol > 
    Denominator = < /L >, < /kgs> (kilograms), < /kgw > (kilograms of water) 
    or 
    < ppt > (part per thousand), < ppm >, < ppb >
    Default = mmol/kgw''') or 'mmol/kgw'
    unit_line = 'unit \t\t ' + solution_units
    
    
    #create the units line of the input file        
    solution_density = input('''What is the solution density? 
    Default = 1 (kg/L)''') or 1
    density_line = 'density \t\t ' + solution_density
    solution_block.append(density_line)
    
    
    #creating the elemental lines of the input file        
    elemental_addition = 'y'
    solution_alkalinity = 'Alkalinity'	CO3-2	 1	Ca0.5(CO3)0.5	50.05   #alkalinity must be parameterized for the simulation
    element_lines = []
    while elemental_addition == 'y':
        if database_selection == 'pitzer':
            pitzer_elements = ['B', 'Ba', 'Br', 'C', 'C(4)', 'Ca', 'Cl', 'E', 
                               'Fe', 'H', 'H(1)', 'K', 'Li', 'Mg', 'Mn', 'Na', 
                               'O', 'O(-2)', 'S', 'S(6)', 'Si', 'Sr']
            pitzer_species = ['B(OH)3', 'Ba+2', 'Br-', 'CO3-2', 'CO3-2', 'Ca+2', 'Cl-', 'e-', 
                              'Fe+2', 'H+', 'H+', 'K+', 'Li+', 'Mg+2', 'Mn+2', 'Na+', 
                              'H2O', 'H2O', 'SO4-2', 'SO4-2', 'H4SiO4', 'Sr+2']
        print('''Element options for the < %s > database:
        < element > code \tcompound''' %(database_selection))
        for possible_element in range(len(pitzer_elements)):
            print(pitzer_elements[possible_element], '\t\t\t', database_species[possible_element])   
        element = input('What element will you simulate?')
        if element not in pitzer_elements:
            print('''The element is not supported by the %s database.
            A Solution_Master must be defined for the %s element''' %(database_selection, element))                       
        element_concentration = input('What is the elemental concentration?')
        special_element_unit = input('''Are the units different than is previously defined?
        < y > or < n >''')
        if special_element_unit == 'y':
            element_unit = input('''What is the element unit?
            numerator = < mg >, < ug >, < mmol >, < umol > 
            Denominator = < /L >, < /kgs> (kilograms), < /kgw > (kilograms of water) 
            or 
            < ppt > (part per thousand), < ppm >, < ppb >
            Default = mmol/kgw''')
        elif special_element_unit == 'n':
            element_unit = solution_units
        elemental_formula_selection = input('''Is the formula different than the provided compound?
        < y > or < n >''')
        if elemental_formula_selection == 'y':
            elemental_formula = input('What is the molecular formula of the compound?')
        elif elemental_formula_selection == 'n':
            elemental_formula = database_species[pitzer_elements.index(element)]

        element_line = '%s\t%s %s\t\tas %s'  %(element, element_concentration, element_unit,elemental_formula)            
        solution_block.append(element_line)

        elemental_addition = input('''Will you parameterize another element?
        < y > or < n >''')
    
    
    #creating the elemental lines of the input file   
    solution_isotope = input('''Are exceptional elemental isotopes used?
    < y > or < n >''')                              
    if solution_isotope == 'y':
        isotope_mass = input('''What is the isotopic mass?
        (amu)''')
        isotope_element = input('''What is the elemental symbol of the isotope?
        e.g. < C > or < Si >''')
    isotope_line = '-isotope\t\t' + isotope_mass + isotope_element
    solution_block.append(isotope_line)


    #defining the mass of water
    water_mass = input('''What is the mass of water in the simulation?
    Default = 1 (kg)''') or 1  
    water_line = '-water \t\t ' + water_mass
    solution_block.append(water_line)

    
    #viewing the SOLUTION block
    printing_solution_block = input('''Would you like to view the entire SOLUTION block?
    < y > or < n >''')
    if printing_solution_block == 'y':
        for element in solution_block:
            print(element)
    
    
    #exporting the SOLUTION block
    exporting_solution_block = input('''Would you like to export the SOLUTION block?
    < y > or < n >''')
    if exporting_solution_block == 'y':
        solution_file = open('%s _PHREEQpy SOLUTION block.txt' %(datetime.now()),'w')
        solution_file.writelines(solution_block)
        solution_file.close()
    
    
    
    #return initial_conditions
               
                        
def make_selected_output(components):
    """
    Build SELECTED_OUTPUT data block
    """
    headings = "-headings    cb    H    O    "
    for i in range(len(components)):
        headings += components[i] + "\t"
    selected_output = """
    SELECTED_OUTPUT
        -reset false
    USER_PUNCH
    """
    selected_output += headings + "\n"
    #
    # charge balance, H, and O
    #
    code = '10 w = TOT("water")\n'
    code += '20 PUNCH CHARGE_BALANCE, TOTMOLE("H"), TOTMOLE("O")\n'
    #
    # All other elements
    #
    lino = 30
    for component in components:
        code += '%d PUNCH w*TOT(\"%s\")\n' % (lino, component)
        lino += 10
    selected_output += code
    return selected_output


def initialize(cells, first=False):
    """
    Initialize IPhreeqc module
    """
    phreeqc = phreeqc_mod.IPhreeqc()                 
    phreeqc.load_database(r"C:\Program Files\USGS\IPhreeqcCOM 3.6.2-15100\database\phreeqc.dat")
    initial_conditions = make_initial_conditions()
    phreeqc.run_string(initial_conditions)
    components = phreeqc.get_component_list()
    selected_output = make_selected_output(components)
    phreeqc.run_string(selected_output)
    phc_string = "RUN_CELLS; -cells 0-1\n"
    phreeqc.run_string(phc_string)
    conc = get_selected_output(phreeqc)
    inflow = {}
    initial = {}
    for name in conc:
        if first:
            inflow[name] = conc[name][0]
        else:
            inflow[name] = conc[name][1]
        initial[name] = conc[name][1]
    task = initial_conditions + "\n"
    task += "COPY solution 1 %d-%d\n" % (cells[0], cells[1])
    task += "COPY exchange 1 %d-%d\n" % (cells[0], cells[1])
    task += "END\n"
    task += "RUN_CELLS; -cells %d-%d\n" % (cells[0], cells[1])
    task += selected_output
    phreeqc.run_string(task)
    conc = get_selected_output(phreeqc)
    for name in conc:
        value = [initial[name]] * len(conc[name])
        conc[name] = value
    return phreeqc, inflow, conc


def advect_step(phreeqc, inflow, conc, cells):
    """Advect by shifting concentrations from previous time step.
    """
    all_names = conc.keys()
    names = [name for name in all_names if name not in ('cb', 'H', 'O')]
    for name in conc:
        # shift one cell
        conc[name][1:] = conc[name][:-1]
        conc[name][0] = inflow[name]
    modify = []
    for index, cell in enumerate(range(cells[0], cells[1] + 1)):
        modify.append("SOLUTION_MODIFY %d" % cell)
        modify.append("\t-cb      %e" % conc['cb'][index])
        modify.append("\t-total_h %f" % conc['H'][index])
        modify.append("\t-total_o %f" % conc['O'][index])
        modify.append("\t-totals")
        for name in names:
            modify.append("\t\t%s\t%f" % (name, conc[name][index]))
    modify.append("RUN_CELLS; -cells %d-%d\n" % (cells[0], cells[1]))
    cmd = '\n'.join(modify)
    phreeqc.run_string(cmd)
    conc = get_selected_output(phreeqc)
    return conc


def get_selected_output(phreeqc):
    """Return calculation result as dict.

    Header entries are the keys and the columns
    are the values as lists of numbers.
    """
    output = phreeqc.get_selected_output_array()
    header = output[0]
    conc = {}
    for head in header:
        conc[head] = []
    for row in output[1:]:
        for col, head in enumerate(header):
            conc[head].append(row[col])
    return conc


def run(ncells, shifts, specie_names):
    """Do one run in one process.
    """
    cells = (1, ncells)
    phreeqc, inflow, conc = initialize(cells, first=True)
    outflow = {}
    for name in specie_names:
        outflow[name] = []
    for _counter in range(shifts):
        # advect
        conc = advect_step(phreeqc, inflow, conc, cells)
        for name in specie_names:
            outflow[name].append(conc[name][-1])
    return outflow


def write_outflow(file_name, outflow):
    """Write the outflow values to a file.
    """
    fobj = open(file_name, 'w')
    header = outflow.keys()
    for head in header:
        fobj.write('%20s' % head)
    fobj.write('\n')
    for lineno in range(len(outflow[head])):
        for head in header:
            fobj.write('%20.17f' % outflow[head][lineno])
        fobj.write('\n')


def main(ncells, shifts):
    """Run different versions with and without multiprocessing
    """

    def measure_time(func, *args, **kwargs):
        """Convinience function to measure run times.
        """
        start = timeit.default_timer()
        result = func(*args, **kwargs)
        return result, timeit.default_timer() - start

    print('Dimensions')
    print('==========')
    print('number of cells:   ', ncells)
    print('number of shifts   ', shifts)
    specie_names = ('Ca', 'Cl', 'K', 'N', 'Na')
    outflow, run_time = measure_time(run, ncells, shifts, specie_names)
    if not os.path.exists('PHREEQpy output'):
        os.mkdir('PHREEQpy output')
    write_outflow('PHREEQpy output/example 11.txt', outflow)
    print('run time:', run_time, ' (s)')

if __name__ == '__main__':
    main(ncells=40, shifts=120)


## Graphical User Interface development

The code logic was incorporated into a GUI through the Tkinter library. Tkinter was selected for being a builtin library of Python that possesses all of the adequate features for the software development. 

In [ ]:
#import statments
import tkinter  
import tkinter.ttk 
from tkinter.tix import *




#develop the skeleton of the interface window
root = Tk() 
root.geometry('1000x600')
root.title("PHREEQreact") 

'''#view_box = Listbox(root, yscrollcommand = scrollbar.set)
scrollbar = Scrollbar(root, orient = VERTICAL)
scrollbar.pack(side = RIGHT, fill = Y)
scrollbar.config(command = view_box.yview)
#scrollbar = ScrolledWindow(root, width = 500, height = 500).pack()'''




#tutorial entries
label = Label(root, text ="Hello World!").pack()
button = Button(root, text = 'Today?', command = root.destroy).place(x = 100, y = 150)
label3 = Label(root, text = 'What is your name?').place(x = 100, y = 80)
user_name = StringVar()
input_information = Entry(root, textvariable = user_name, width = 60).place(x = 100, y = 100)
print(input_information)


#mock PHREEQreact components 
'''label2 = Label(root, text = "Do you know the thickness of each layer in the RO composite membrane?").place(x = 200, y = 180)
v = StringVar(root, 'membrane_specifications')
options = {'yes':'y', 'no':'n'}
i = 0
for (text, value) in options.items():
    Radiobutton(root, 
                text = text, 
                variable = v,
                value = value).place(x = 200, y = 200 + i)
    i += 20'''

label2 = Label(root, text = "Which RO module will you simulate?").place(x = 200, y = 280)
module_selection = StringVar()
combobox = ttk.Combobox(root, width = 20, textvariable = module_selection)
combobox.place(x = 200, y = 300)
combobox['values'] = ('BW30-400', 'custom')
combobox.current(0)

next_iteration = 0

#save button feature
def next():
    global next_iteration
    global membrane_thickness
    global feed_thickness
    global permeate_thickness
    global polysulfonic_layer_thickness
    global support_layer_thickness
    
    if next_iteration == 0:
        
        module = module_selection.get()
        name = user_name.get()


        #incorporate the BW30-400 features into the simulation
        if module == 'BW30-400':
            membrane_thickness = 250 / nm_over_mm
            feed_thickness = 0.7112 
            permeate_thickness = 0.3 
            polysulfonic_layer_thickness = 0.05 
            support_layer_thickness = 0.15

        elif module == 'custom':
            membrane_thickness = IntVar()
            l1 = Label(root, text = "Filtration membrane thickness (nm)").place(x = 450, y = 200)
            membrane_thickness = Entry(root, textvariable = membrane_thickness).place(x = 650, y = 200)

            feed_thickness =  IntVar()
            l2 = Label(root, text = "Feed spacer thickness (mm)").place(x = 450, y = 220)
            feed_thickness = Entry(root, textvariable = feed_thickness).place(x = 650, y = 220)

            permeate_thickness =  IntVar()
            l3 = Label(root, text = "Permeate spacer thickness (mm)").place(x = 450, y = 240)
            permeate_thickness = Entry(root, textvariable = permeate_thickness).place(x = 650, y = 240)

            polysulfonic_layer_thickness =  IntVar() 
            l4 = Label(root, text = "Polysulfonic layer thickness (mm)").place(x = 450, y = 260)
            polysulfonic_layer_thickness = Entry(root, textvariable = polysulfonic_layer_thickness).place(x = 650, y = 260)

            support_layer_thickness =  IntVar()
            l5 = Label(root, text = "Support layer thickness (mm)").place(x = 450, y = 280)
            support_layer_thickness = Entry(root, textvariable = support_layer_thickness).place(x = 650, y = 280)
        
        next_iteration += 1
        return
    
    
    if next_iteration == 1:
        repeated_membrane_thickness = membrane_thickness + feed_thickness + permeate_thickness + polysulfonic_layer_thickness + support_layer_thickness
        thickness_label = Label(root, text = "%s" %(repeated_membrane_thickness)).place(x = 450, y = 300)
        
        
    
#executing and viewing the interface window
next_button = Button(root, text = 'Next', command = next).pack(side = BOTTOM)

root.mainloop() 

## Creation of PHREEQC input files

An interface for creating PHREEQC input files is coded. The code creates the DATABASE, TITLE, SOLUTION, EQUILIBRIUM_PHASES, REACTION, SELECTED_OUTPUT, and TRANSPORT code blocks. The transport calculations from our excel file are included. The code is designed for user creeation of PHREEQC input files for arbitrary desalination systems and for direct execution in iPHREEQC software.

In [6]:
#import libraries
import datetime
import math
import os
import re


#unit conversions 
mm_over_m = 1E3 
mm_over_inch = 25.4
nm_over_mm = 1E6
grams_over_kilograms = 1E3
hours_over_day = 24
minutes_over_hour = 60
seconds_over_minute = 60
seconds_over_day = hours_over_day * minutes_over_hour * seconds_over_minute
liters_over_cubic_meter = 1E3


#calculation constants
grams_over_liters_h2o = 997.07 # @25 degrees celcius
grams_over_moles_h2o = 18.015 
kinematic_flow_velocity = 9.33E-7    #square meters / second
simulated_time_over_computational_time = 9.29    


#simulation constants
timesteps_over_cell = 1

#useful conditions and parameters
possible_answers = ['y', 'n']



def make_general_conditions():
    """
    Specify general information of the simulation
    """
    global database_selection
    global general_conditions
    print('We will parameterize your path directory for the iPHREEQC software.')
    computer_drive_path = input('''What is your computer drive directory?
    Default = C:''') or 'C:\\'
    program_path = input('''What is the program directory that stores iPHREEQC?
    Default = Program Files (x86)''') or 'Program Files (x86)'
    folder_path = input('''What is the folder that contains iPHREEQC software?
    Default = USGS''') or 'USGS'
    software_path = input('''What is the folder of iPHREEQC?
    Default = Phreeqc Interactive 3.6.2-15100''') or 'Phreeqc Interactive 3.6.2-15100'
    iphreeqc_path = os.path.join(computer_drive_path, program_path, folder_path, software_path)
    print(iphreeqc_path)
    while not os.path.exists(iphreeqc_path):
        print('''ERROR: The entered directory does not exist on your computer. 
        Please reconstitute the iPHREEQC directory.''')
        computer_drive_path = input('''What is your computer drive directory?
        Default = C:''') or 'C:\\'
        program_path = input('''What is the program directory that stores iPHREEQC?
        Default = Program Files (x86)''') or 'Program Files (x86)'
        folder_path = input('''What is the folder that contains iPHREEQC software?
        Default = USGS''') or 'USGS'
        software_path = input('''What is the folder of iPHREEQC?
        Default = Phreeqc Interactive 3.6.2-15100''') or 'Phreeqc Interactive 3.6.2-15100'
        iphreeqc_path = os.path.join(computer_drive_path, program_path, folder_path, software_path)
        print(iphreeqc_path)
    databases = ['pitzer', 'phreeqc', 'Amm', 'ColdChem', 'core10', 'frezchem', 'iso', 'llnl', 
                 'minteq', 'minteq.v4', 'sit', 'Tipping_Hurley', 'wateq4f']
    described_databases = ['pitzer', 'phreeqc']

    for database in databases:
        print('< %s >\t' %(database))
    database_selection = input('What database do you select?')
    while database_selection not in databases:
        print('''ERROR: The entry is beyond the accepted values. 
        Please provide an accepted value''')
        database_selection = input('What database do you select?')
    while database_selection not in described_databases:
        print('''The database is not current defined in the this interface.
        Only < pitzer > and < phreeqc > are defined by the code.''')    
        database_selection = input('What database do you select?')
    database_path = iphreeqc_path + '\\database\\%s.dat' %(database_selection)
    simulation_title = input('What is the title of your simulation?')   
    database_line = 'DATABASE %s' %(database_path)
    title_line = 'TITLE\t %s' %(simulation_title)

    general_conditions = [database_line, title_line]
        
        
        
def make_solutions():
    """
    Specify the SOLUTION block of the simulation.
    """
    global solution_block
    global elements
    
    if database_selection == 'pitzer':
        database_elements = ['Alkalinity', 'Alkalinity', 'B', 'Ba', 'Br', 'C', 'C(4)', 'Ca', 'Cl', 'E', 
                   'Fe', 'H', 'H(1)', 'K', 'Li', 'Mg', 'Mn', 'Na', 
                   'O', 'O(-2)', 'S', 'S(6)', 'Si', 'Sr']
        database_species = ['HCO3', 'CaCO3', 'B(OH)3', 'Ba+2', 'Br-', 'CO3-2', 'CO3-2', 'Ca+2', 'Cl-', 'e-', 
                  'Fe+2', 'H+', 'H+', 'K+', 'Li+', 'Mg+2', 'Mn+2', 'Na+', 
                  'H2O', 'H2O', 'SO4-2', 'SO4-2', 'H4SiO4', 'Sr+2']
        
    elif database_selection == 'phreeqc':
        print('''The database is under construction.
        Please select the < pitzer > database.''')
        database_elements = []
        database_species = []
    
    solution_block = []
    #create the solution line of the input file
    print('''We will now create the SOLUTION block. 
=================''')
    solution_description = input('''What is a description of your feed solution?
    Default is none\t''')
    
    initial_solution_line = '\nSOLUTION 0\t%s' % (solution_description)
    solution_block.append(initial_solution_line)

    
    #=============================================================================
    #determine which predefined solution should be simulated
    water_selections = ['Red Sea', 'Mediterranean Sea', 'German Basin', 'Marcellus Appalachian Basin', 'Bekken Formation', 
                          'China Sea', 'Michigan Basin', 'Palo Duro Basin', 'Western Pennsylvannia Basin', 'Custom']
    
    defined_waters = ['Red Sea', 'Custom']
    print('\n')    
    for selection in water_selections:
        print('< %s >' %(selection))
    water_selection = input('Which water geochemistry would you like to simulate?')
    while water_selection not in water_selections:
        print('''ERROR: The entry is beyond the accepted values. 
        Please provide an accepted value''')
        water_selection = input('Which water geochemistry would you like to simulate?')
    while water_selection not in defined_waters:
        print('''The water selection has not yet been parameterized in the source code. 
        The current options are < Red Sea > and < Custom >.''')
        water_selection = input('Which water geochemistry would you like to simulate?')
        
    if water_selection == 'Red Sea':
        elements = ['Mn', 'Fe', 'B', 'Cl', 'Na', 'S(6)', 'Ca', 'K', 'Mg', 'Sr', 'Ba', 'Li']
        red_sea_concentrations = ['0.000306', '0.006281', '1.344', '24756', '16417.2', '9500', '774', '301',
                                 '1646', '8.3', '0.011', '0.228']
        red_sea_units = []
        i = 0
        while i <= len(elements):
            red_sea_units.append('ppm')
            i += 1
        element_comments = ['#average of Al-Taani et al., 2014 and Longinelli and Craig, 1967.', '#Sabine et al., 2002 as reported for the Indian Ocean',
                           '#Al-Taani et al., 2014 // 4.00 is the default (?)', '#Al-Taani et al., 2014 for the Northern Gulf of Aqaba',
                           '#Al-Taani et al., 2014 for the Northern Gulf of Aqaba', '#Al-Taani et al., 2014 ',
                           '#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967',
                           '#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967 describes [Na]=15834',
                           '#Longinelli and Craig, 1967 and Llyod, 1967.', '#Abdel-Aal et al., 2015', '#Abdel-Aal et al., 2015',
                           '#Abdel-Aal et al., 2015', '#Bernat, Church, and Allegre, 1972 from the Mediterranean',
                           '#Bernat, Church, and Allegre, 1972 from the Mediterranean', '#Stoffyn-Egli and Mackenzie, 1984 for the Mediterranean Sea']
        
        
        #parameterize solution temperature
        temperature_line = 'temp \t 24.5 \t #average of Al-Taani et al., 2014 and Longinelli and Craig, 1967.'
        solution_block.append(temperature_line)
    
        #parameterize solution ph
        ph_line = 'pH \t 8.22'
        solution_block.append(ph_line) 

        #parameterize solution pe
        pe_line = 'pe \t 0.2679 \t   #Al-Taani et al., 2014 // 4.00 is the default (?)'
        solution_block.append(pe_line)
        
        #parameterize solution concentration units
        solution_unit = 'ppm'
        unit_line = 'units \t %s' %(solution_unit)
        solution_block.append(unit_line)
        
        '''
        #parameterize the concentration units
        unit_line = 'unit\t ppm'
        solution_block.append(unit_line)
        '''
        #parameterize the elemental concentrations
       
        for element in elements:
            element_concentration = red_sea_concentrations[elements.index(element)]
            element_unit = red_sea_units[elements.index(element)]
            elemental_formula = database_species[database_elements.index(element)]
            element_comment = element_comments[elements.index(element)]
            
            if len(element_concentration) > 3:
                element_line = '%s\t%s %s\tas %s\t\t%s'  %(element, element_concentration, element_unit,
                                                           elemental_formula,element_comment)             
            elif len(element_concentration) <= 3:
                element_line = '%s\t%s %s\t\tas %s\t\t%s'  %(element, element_concentration, element_unit,
                                                           elemental_formula,element_comment)             
            solution_block.append(element_line)
            
            
        #parameterize the inital water mass
        water_mass = feed_mass_cell
        water_line = '-water \t%s' %(water_mass)
        solution_block.append(water_line)

            
        #=========================================================================================   
        #parameterize the initial module solution
        feed_solution_line = '\nSOLUTION 1-%s\tInitial solution in the RO module' %(cells_per_module) 
        solution_block.append(feed_solution_line)
        solution_block.append('temp \t 25')
        solution_block.append('units \t ppm')
        
        for element in elements:
            element_concentration = 0
            element_line = '%s\t%s'  %(element, element_concentration)    
            solution_block.append(element_line)
            
        solution_block.append(water_line)
            
    
    elif water_selection == 'Custom':
    
        #==============================================================================
        #create the temperature line of the input file
        solution_temperature = input('''What is the solution temperature? 
        Default = 25 (K)\t''') or str(25)
        temperature_line = 'temp \t %s' %(solution_temperature)
        solution_block.append(temperature_line)


        #==============================================================================
        #create the pH line of the input file
        solution_ph = input('What is the solution pH?')
        error = 'yes'
        while error == 'yes':
            try:
                float(solution_ph) == True
                error = 'no'
            except:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                solution_ph = input('What is the solution pH?')
        ph_charge_balance = input('''Should the pH be charge balanced?
        < y > or < n >\t''')
        if ph_charge_balance == 'y':
            solution_ph += '\tcharge'
        while ph_charge_balance not in ('n', 'y'):
            print('''ERROR: The entry is beyond the accepted values. 
            Please provide an accepted value''')
            ph_charge_balance = input('''Should the pH be charge balanced?
            < y > or < n >\t''')

        ph_line = 'pH \t %s' %(solution_ph)
        solution_block.append(ph_line)


        #==============================================================================
        #create the pe line of the input file
        solution_pe = input('''What is the solution pe (-log(electron activity))? 
        Default = 4.0\t''') or str(4)
        while not solution_pe.isnumeric():
            print('''ERROR: The entry is beyond the accepted values. 
            Please provide an accepted value''')
            solution_pe = input('''What is the solution pe (-log(electron activity))? 
            Default = 4.0\t''') or str(4)

        pe_charge_balance = input('''Should the pe phase be balanced? 
            < y > or < n > ; Default = < n >\t''') or 'n'
        pe_charge = ''
        if pe_charge_balance == 'y':
            pe_phase = input('What is the balanced phase name?')
            pe_phase_index = input('''What is the phase saturation index?
            Default = 0\t''') or str(0)
            pe_charge = ' %s %s' %(pe_phase, pe_phase_index) 

        pe_line = 'pe \t ' + solution_pe + pe_charge
        solution_block.append(pe_line)


        #==============================================================================
        #create the redox line of the input file      
        solution_redox = input('''Is a special pe calculated via a redox reaction? 
        < y > or < n > ; Default = < n >\t''') or 'n'
        if solution_redox == 'y':
            redox_couple = input('''What is the redox couple?
            e.g. O(-2)/O(0) for oxygen with (-2) and (0) oxidation states\t''')
            redox_line = 'redox \t\t ' + redox_couple
            solution_block.append(redox_line)


        #==============================================================================
        #create the units line of the input file        
        unit_numerators = ['mg', 'ug', 'mmol', 'umol'] 
        unit_denominators = ['L', 'kgs', 'kgw'] 
        unit_concentration = ['ppt', 'ppm', 'ppb']
        print('Numerators:')
        for unit in unit_numerators:
            print('< %s >' %(unit)) 
        print('Denominators:')
        for unit in unit_denominators:
            print('< %s >' %(unit)) 
        print('Concentrations:')
        for unit in unit_concentration:
            print('< %s >' %(unit))
        solution_units = input('''What are the concentration units of your solution?
        Default = mmol/kgw\t''') or 'mmol/kgw'

        if re.search('(\/)', solution_units):
            numerator, denominator = solution_units.split('/')
            while numerator not in unit_numerators or denominator not in unit_denominators:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                solution_units = input('What are the concentration units of your solution?')

        unit_line = 'unit\t ' + solution_units
        solution_block.append(unit_line)


        #==============================================================================
        #create the units line of the input file        
        solution_density = input('''What is the solution density? 
        Default = 1 (kg/L)\t''') or str(1)
        while not solution_pe.isnumeric():
            print('''ERROR: The entry is beyond the accepted values. 
            Please provide an accepted value''')
            solution_density = input('''What is the solution density? 
            Default = 1 (kg/L)\t''') or str(1)

        density_line = 'density\t ' + solution_density
        solution_block.append(density_line)


        #==============================================================================
        #creating the elemental lines of the input file     

    

        elemental_addition = 'y'
        element_lines = []
        while elemental_addition == 'y':
            print('''Element options for the < %s > database: \n< element >\t\tcompound''' %(database_selection))
            for possible_element in database_elements:
                if possible_element not in ('Alkalinity', 'C(4)', 'H(1)', 'O(-2)', 'S(6)'):
                    print('< %s >' %(possible_element), '\t\t\t', database_species[database_elements.index(possible_element)]) 
                if possible_element in ('Alkalinity', 'C(4)', 'H(1)', 'O(-2)', 'S(6)'):
                    print('< %s >' %(possible_element), '\t\t', database_species[database_elements.index(possible_element)])                 
            element = input('''What element will you simulate?
            Type < done > when you have finished parameterizing elements.''')
            if element == 'done':
                elemental_addition = 'n'
            while element not in database_elements:
                print('''The element is not supported by the %s database.
                A Solution_Master must be defined for the %s element''' %(database_selection, element))  
                element = input('What element will you simulate?\t')

            element_concentration = input('What is the elemental concentration?')
            while not element_concentration.isnumeric():
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                element_concentration = input('What is the elemental concentration?')
                
            special_element_unit = input('''Are the units different than is previously defined?
            < y > or < n > ; Default = < n >\t''') or 'n'
            if special_element_unit == 'y':
                element_unit = input('''What is the element unit?
                numerator = < mg >, < ug >, < mmol >, < umol > 
                Denominator = < /L >, < /kgs> (kilograms), < /kgw > (kilograms of water) 
                or 
                < ppt > (part per thousand), < ppm >, < ppb >
                Default = mmol/kgw\t''')
            elif special_element_unit == 'n':
                element_unit = ''
                
            elemental_formula_selection = input('''Is the formula different than the provided compound?
            < y > or < n > ; Default = < n >\t''') or 'n'
            if elemental_formula_selection == 'y':
                elemental_formula = input('What is the molecular formula of the compound?\t')
            elif elemental_formula_selection == 'n':
                elemental_formula = database_species[database_elements.index(element)]
                
            possible_elemental_comment = input('''Does the elemental information have associated commentary or a reference?
            < y > or < n >''')
            element_comment = ''
            if possible_elemental_comment == 'y':
                elemental_comment = input('What is the comment or reference?')
                element_comment = '#%s' %(elemental_comment)
            elif possible_elemental_comment == 'n':
                pass
            else:
                while possible_elemental_comment not in ('y' or 'n'):
                    print('''ERROR: The entry is beyond the accepted values. 
                    Please provide an accepted value''')
                    possible_elemental_comment = input('''Does the elemental information have associated commentary or a reference?
                    < y > or < n >''')
                
            if element == 'Alkalinity':
                element_line = '%s\t%s %s\tas %s\t\t%s'  %(element, element_concentration, element_unit,
                                                     elemental_formula, element_comment)            
                solution_block.append(element_line)
            else:
                element_line = '%s\t\t%s %s\tas %s\t\t%s'  %(element, element_concentration, element_unit,
                                                       elemental_formula, element_comment)            
                solution_block.append(element_line)

    
    
        #==============================================================================
        #creating the elemental lines of the input file   
        solution_isotope = input('''Are exceptional elemental isotopes used?
        < y > or < n > ; Default = < n >\t''') or 'n'                      
        if solution_isotope == 'y':
            isotope_mass = input('''What is the isotopic mass?
            (amu)\t''')
            isotope_element = input('''What is the elemental symbol of the isotope?
            e.g. < C > or < Si >\t''')
            isotope_line = '-isotope\t' + isotope_mass + isotope_element
            solution_block.append(isotope_line)


        #==============================================================================
        #defining the mass of water
        water_mass = input('''What is the mass of water in the simulation?
        Default = 1 (kg)\t''') or str(1)  
        while not water_mass.isnumeric():
            print('''ERROR: The entry is beyond the accepted values. 
            Please provide an accepted value''')
            water_mass = input('''What is the mass of water in the simulation?
            Default = 1 (kg)\t''') or str(1)  

        water_line = '-water \t ' + water_mass
        solution_block.append(water_line)


        #looping to the next element in the simulation
        elemental_addition = input('''Will you parameterize another element?
        < y > or < n >\t''')
        


    
def make_equilibrium_phases():
    """
    Specify the EQUILIBRIUM_PHASES block of the simulation.
    """
    global equilibrium_phases_block
    global minerals
    if database_selection == 'pitzer':
        pitzer_minerals = ['Akermanite', 'Anhydrite', 'Anthophyllite', 'Antigorite', 'Aragonite', 'Arcanite', 'Artinite',
                           'Barite', 'Bischofite', 'Bloedite', 'Brucite', 'Burkeite', 'Calcite', 'Carnallite', 'Celestite',
                           'Chalcedony', 'Chrysotile', 'Diopside', 'Dolomite', 'Enstatite', 'Epsomite', 'Forsterite',
                           'Gaylussite', 'Glaserite', 'Glauberite', 'Goergeyite', 'Gypsum', 'Halite', 'Hexahydrite', 'Huntite', 'Kainite',
                           'Kalicinite', 'Kieserite', 'Labile_S', 'Leonhardite', 'Leonite', 'Magnesite', 'MgCl2_2H2O',
                           'MgCl2_4H2O', 'Mirabilite', 'Misenite', 'Nahcolite', 'Natron', 'Nesquehonite', 'Pentahydrite', 
                           'Pirssonite', 'Polyhalite', 'Portlandite', 'Quartz', 'Schoenite', 'Sepiolite(d)', 'Sepiolite', 
                           'SiO2(a)', 'Sylvite','Syngenite', 'Talc', 'Thenardite', 'Trona', 'Borax', 'Boric_acid,s', 
                           'KB5O8:4H2O', 'K2B4O7:4H2O', 'NaBO2:4H2O', 'NaB5O8:5H2O', 'Teepleite']
        pitzer_mineral_formulas = ['Ca2Mg[Si2O7]', 'CaSO4', '☐Mg2Mg5Si8O22(OH)2', 'Mg48Si34O85(OH)62', 'CaCO3', 'K2SO4', 'Mg2(CO3)(OH)2·3H2O',
                                  'BaSO4', 'MgCl2·6H2O', 'Na2Mg(SO4)2·4H2O', 'Mg(OH)2', 'Na6(CO3)(SO4)2', 'CaCO3', 'KMgCl3•6(H2O)', 'SrSO4',
                                  'SiO2', 'Mg3Si2O5(OH)4', 'CaMgSi2O6', 'CaMg(CO3)2', 'MgSiO3', 'MgSO4•7(H2O)', 'Mg2SiO4',
                                  'Na2Ca(CO3)2•5(H2O)', 'NaK3(SO4)2', 'Na2Ca(SO4)2', 'K2Ca5(SO4)6•(H2O)', 'CaSO4•2(H2O)', 'NaCl', 'MgSO4•6(H2O)', 'CaMg3(CO3)4', 'MgSO4•KCl•3(H2O)',
                                  'KHCO3', 'MgSO4•(H2O)', 'Na4Ca(SO4)3•2H2O', 'MgSO4•4(H2O)', 'K2Mg(SO4)2•4(H2O)', 'MgCO3', 'MgCl2:2H2O',
                                  'MgCl2•4H2O', 'Na2SO4•10(H2O)', 'K8H6(SO4)7', 'NaHCO3', 'Na2CO3•10(H2O)', 'Mg(HCO3)(OH)•2(H2O)', 'MgSO4•5(H2O)',
                                  'Na2Ca(CO3)2•2(H2O)', 'K2Ca2Mg(SO4)4•2(H2O)', 'Ca(OH)2', 'SiO2', 'K2Mg(SO4)2•6(H2O)', 'Mg4Si6O15(OH)2•6(H2O)', 'Mg4Si6O15(OH)2•6(H2O)',
                                  'SiO2', 'KCl', 'K2Ca(SO4)2•(H2O)', 'Mg3Si4O10(OH)2', 'Na2SO4', 'Na3(CO3)(HCO3)•2(H2O)', 'Na2B4O5(OH)4•8(H2O)', 'B(OH)3',
                                  'KB5O8•4H2O', 'K2B4O7•4H2O', 'NaBO2•4H2O', 'NaB5O8•5H2O', 'Na2B(OH)4Cl']     

        #print(len(pitzer_minerals))
        #print(len(pitzer_mineral_formulas))
    #============================================================================
    #create the initial line of the equilibrium phases block
    equilibrium_phases_block = []
    equilibrium_phases_number = input('''We will now create the EQUILIBRIUM_PHASES block. 
    =================
    To what cell(s) does this solution apply? 
    < # > or < #-# > ; Default = 1 - %s\t''' %(cells_per_module)) or '1-%s' %(cells_per_module)
    equilibrium_phases_description = input('''What is the description of the block?
    Default is none\t''')
    equilibrium_phases_line = '\nEQUILIBRIUM_PHASES %s\t%s' % (equilibrium_phases_number, equilibrium_phases_description)
    equilibrium_phases_block.append(equilibrium_phases_line)

        
    #=============================================================================
    #determine which predefined set of minerals should be simulated
    mineral_selections = ['All', 'All but a few', 'Build from scratch']
    print('''\nMineral options for the < %s > database: \nmineral\t\t\tcompound''' %(database_selection))  
    print('====================================')
    short_mineral_names = ['Barite', 'Gypsum', 'Halite', 'Natron', 'Quartz', 'Talc','Trona', 'Borax']
    for possible_mineral in pitzer_minerals:
        if possible_mineral in short_mineral_names:
            print('%s' %(possible_mineral), '\t\t\t', pitzer_mineral_formulas[pitzer_minerals.index(possible_mineral)])
        elif possible_mineral not in short_mineral_names:
            print('%s' %(possible_mineral), '\t\t', pitzer_mineral_formulas[pitzer_minerals.index(possible_mineral)])  
    for selection in mineral_selections:
        print('\n< %s >' %(selection))
        
    mineral_selection = input('Which selection of minerals would you like to simulate?')
    while mineral_selection not in mineral_selections:
        print('''ERROR: The entry is beyond the accepted values. 
        Please provide an accepted value''')
        mineral_selection = input('Which selection of minerals would you like to simulate?')
        
        
    #=============================================================================
    #establish and edit all minerals for a simulation
    if mineral_selection == 'All':
        minerals = pitzer_minerals
        mineral_saturation_states = []
        minerals_initial_moles = []
        i = 0
        while i <= len(minerals):
            mineral_saturation_states.append(0)
            minerals_initial_moles.append(0)
            i += 1
            
        possible_saturation_states = input('''Do any minerals possess a non-zero saturation index?
        < y > or < n >''')
        possible_options = ['y', 'n']
        if possible_saturation_states == 'y':
            print('\nYour selected minerals:')
            for mineral in minerals:
                print('< %s >' %(mineral))
            nonzero_mineral_saturation = input('Which mineral possesses an existing saturation index?')    
            mineral_saturation_state = input('What is the saturation index?')
            mineral_saturation_states[minerals.index(nonzero_mineral_saturation)] = mineral_saturation_state

        elif possible_saturation_states == 'n':
            pass
            
        else:
            while possible_saturation_states not in possible_options:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                possible_saturation_states = input('''Do any minerals possess a non-zero saturation index?
                < y > or < n >''')

            
        preexisting_minerals = input('''Do any minerals currently previously exist in the module?
        < y > or < n >''')
        if preexisting_minerals == 'y':
            print('\nYour selected minerals:')
            for mineral in minerals:
                print(mineral)
            existing_mineral = input('Which mineral possesses an existing saturation index?')       
            mineral_inital_moles = input('What is the saturation index?')
            minerals_initial_moles[minerals.index(existing_mineral)] = mineral_inital_moles
            
        elif preexisting_minerals == 'n':
            pass
            
        else:
            while preexisting_minerals not in possible_options:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                preexisting_minerals = input('''Do any minerals currently previously exist in the module?
                < y > or < n >''')
    
        #add the elemental lines to the EQUILIBRIUM_PHASES block of the code
        short_mineral_names = ['Barite','Brucite', 'Calcite', 'Gypsum','Leonite', 'Halite','Huntite', 'Natron', 'Quartz', 
                               'SiO2(a)', 'Talc','Trona', 'Borax', 'Kainite', 'Sylvite']
        for mineral in range(len(minerals)):
            if minerals[mineral] in short_mineral_names:
                equilibrium_phases_line = '%s\t\t%s\t%s' %(minerals[mineral], mineral_saturation_states[mineral], 
                                                         minerals_initial_moles[mineral])
            elif minerals[mineral] not in short_mineral_names:
                equilibrium_phases_line = '%s\t%s\t%s' %(minerals[mineral], mineral_saturation_states[mineral], 
                                                         minerals_initial_moles[mineral]) 
        
            
            equilibrium_phases_block.append(equilibrium_phases_line)  
            

    #=============================================================================
    #establish and systematically remove all but a few minerals for simulation
    if mineral_selection == 'All but a few':
        minerals = pitzer_minerals
        mineral_saturation_states = []
        minerals_initial_moles = []
        i = 0
        while i <= len(minerals):
            mineral_saturation_states.append(0)
            minerals_initial_moles.append(0)
            i += 1
        print('\nAll possible minerals with the %s database:' %(database_selection))
        for mineral in minerals:
            print('< %s >' %(mineral))
        
        removed_mineral = input('''Which mineral should be removed?
            One must be selected at a time.
            Enter < done > when you are finished.''')
        while removed_mineral != 'done':
            while removed_mineral != 'done' and removed_mineral not in minerals:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                removed_mineral = input('''Which mineral should be removed?
                Only one can be removed at a time.
                Enter < done > when you are finished.''')

            removed_mineral_index = minerals.index(removed_mineral)
            del minerals[removed_mineral_index], mineral_saturation_states[removed_mineral_index], minerals_initial_moles[removed_mineral_index]
            
            removed_mineral = input('Which mineral should be removed?')

            
        #include non-zero saturation indices    
        possible_saturation_states = input('''Do any minerals possess a non-zero saturation index?
        < y > or < n >''')
        possible_options = ['y', 'n']
        if possible_saturation_states == 'y':
            print('\nYour selected minerals:')
            for mineral in minerals:
                print('< %s >' %(mineral))
            nonzero_mineral_saturation = input('''Which mineral possesses an existing saturation index?
            One must be selected at a time.
            Enter < done > when you are finished.''')    
            while nonzero_mineral_saturation != 'done':
                while removed_mineral != 'done' and nonzero_mineral_saturation not in minerals:
                    print('''ERROR: The entry is beyond the accepted values. 
                    Please provide an accepted value''')
                    removed_mineral = input('''Which mineral possesses an existing saturation index?
                    Only one can be removed at a time.
                    Enter < done > when you are finished.''')
                mineral_saturation_state = input('What is the saturation index?')
                mineral_saturation_states[minerals.index(nonzero_mineral_saturation)] = mineral_saturation_state
                nonzero_mineral_saturation = input('Which mineral possesses an existing saturation index?')
                
        elif possible_saturation_states == 'n':
            pass
            
        else:
            while possible_saturation_states not in possible_options:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                possible_saturation_states = input('''Do any minerals possess a non-zero saturation index?
                < y > or < n >''')

                
        #include previously existing mineral moles
        preexisting_minerals = input('''Do any minerals previously exist in the module?
        < y > or < n >''')
        if preexisting_minerals == 'y':
            print('\nYour selected minerals:')
            for mineral in minerals:
                print(mineral)
            existing_mineral = input('''Which mineral possesses an existing saturation index?
            One must be selected at a time.
            Enter < done > when you are finished.''')  
            while existing_mineral != 'done':
                while existing_mineral != 'done' and existing_mineral not in minerals:
                    print('''ERROR: The entry is beyond the accepted values. 
                    Please provide an accepted value''')
                    removed_mineral = input('''Which mineral possesses an existing saturation index?
                    Only one can be removed at a time.
                    Enter < done > when you are finished.''')
                mineral_inital_moles = input('What is the saturation index?')
                minerals_initial_moles[minerals.index(existing_mineral)] = mineral_inital_moles
                existing_mineral = input('Which mineral possesses an existing saturation index?')     
            
        elif preexisting_minerals == 'n':
            pass
            
        else:
            while preexisting_minerals not in possible_options:
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                preexisting_minerals = input('''Do any minerals previously exist in the module?
                < y > or < n >''')
            
            
        #add the elemental lines to the EQUILIBRIUM_PHASES block of the code
        short_mineral_names = ['Barite','Brucite', 'Calcite', 'Gypsum','Leonite', 'Halite','Huntite', 'Natron', 'Quartz', 
                               'SiO2(a)', 'Talc','Trona', 'Borax', 'Kainite', 'Sylvite']
        for mineral in range(len(minerals)):
            if minerals[mineral] in short_mineral_names:
                equilibrium_phases_line = '%s\t\t%s\t%s' %(minerals[mineral], mineral_saturation_states[mineral], 
                                                         minerals_initial_moles[mineral])
            elif minerals[mineral] not in short_mineral_names:
                equilibrium_phases_line = '%s\t%s\t%s' %(minerals[mineral], mineral_saturation_states[mineral], 
                                                         minerals_initial_moles[mineral]) 
        
            
            equilibrium_phases_block.append(equilibrium_phases_line)        
        
        
    #=============================================================================
    #establish a simulation with user-defined minerals
    if mineral_selection == 'Build from scratch':
        minerals = pitzer_minerals
        mineral_saturation_states = []
        minerals_initial_moles = []
        
        if database_selection == 'pitzer':
            print('''Mineral options for the < %s > database: \n< mineral >\t\tcompound''' %(database_selection))
            long_mineral_names = ['Anthophyllite', 'Hexahydrite', 'Leonhardite', 'Nesquehonite', 'Pentahydrite', 'Portlandite',
                                 'Sepiolite(d)', 'Boric_acid,s', 'K2B4O7:4H2O', 'NaB5O8:5H2O']
            for possible_mineral in pitzer_minerals:
                if possible_mineral in long_mineral_names:
                    print('< %s >' %(possible_mineral), '\t', pitzer_mineral_formulas[pitzer_minerals.index(possible_mineral)])
                elif possible_mineral not in long_mineral_names:
                    print('< %s >' %(possible_mineral), '\t\t', pitzer_mineral_formulas[pitzer_minerals.index(possible_mineral)])  

        mineral_addition = 'y'
        while mineral_addition == 'y':
            mineral = input('What mineral will you simulate?')
            while mineral not in pitzer_minerals:
                print('''The mineral is not supported by the %s database.
                A Solution_Master must be defined for the %s mineral''' %(database_selection, mineral))  
                mineral = input('What mineral will you simulate?\t')

            minerals.append(mineral)

            #include non-zero saturation indices    
            mineral_saturation_state = input('''What is the saturation index of the mineral?
            Default = 0''') or str(0)
            while not mineral_saturation_state.isnumeric():
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                removed_mineral = input('''What is the saturation index of the mineral?
                Default = 0''')
           
            #include previously existing mineral moles
            preexisting_mineral_moles = input('''What is the existing quantity of moles for the mineral in the module?
            Default = 0''') or str(0)
            while not preexisting_mineral_moles.isnumeric():
                print('''ERROR: The entry is beyond the accepted values. 
                Please provide an accepted value''')
                preexisting_mineral_moles = input('''What is the existing quantity of moles for the mineral in the module?
                Default = 0''') or str(0)
                
            #add the mineral information to the EQUILIBRIUM_PHASES block
            short_mineral_names = ['Barite','Brucite', 'Calcite', 'Gypsum','Leonite', 'Halite','Huntite', 'Natron', 'Quartz', 
                                   'SiO2(a)', 'Talc','Trona', 'Borax', 'Kainite', 'Sylvite']
            if mineral in short_mineral_names:
                equilibrium_phases_line = '%s\t\t%s\t%s' %(mineral, mineral_saturation_state, preexisting_mineral_moles)
            elif mineral not in short_mineral_names:
                equilibrium_phases_line = '%s\t%s\t%s' %(mineral, mineral_saturation_state, preexisting_mineral_moles) 
                
            equilibrium_phases_block.append(equilibrium_phases_line)
                
            #loop to the next 
            mineral_addition = input('''Will you add another mineral to the simulation?
            < y > or < n >''')
            
            


def make_reactive_transport():
    '''
    Specify the REACTION and TRANSPORT blocks of PHREEQC simulations
    '''
    global permeate_removal_per_cell
    global feed_mass_cell
    global cells_per_module
    global reaction_block
    global transport_block
    global time_or_distance
    
    #===========================================================
    #parameterize the module 
    possible_ro_modules = ['BW30-400', 'Custom']
    print('We will now parameterize the REACTION block.')
    for module in possible_ro_modules:
        print('< %s >' %(module))
    module_selection = input('What RO module will you simuate?')
    while module_selection not in possible_ro_modules:
        print('''The module selection is not supported by this simulation.
        Select one of the options.''')  
        module_selection = input('What RO module will you simuate?')
                                 
    if module_selection == 'BW30-400':
        module_diameter =  201                   #mm
        permeate_tube_diameter =  29             #mm
        module_length =  1.016                   #m
        permeate_flow = 40                       #cubic meters / day
        max_feed_flow = 15.9                     #cubic meters / hour
        membrane_thickness = 250 / nm_over_mm    #nm
        feed_thickness = 0.7112                  #mm
        permeate_thickness = 0.3                 #mm
        polysulfonic_layer_thickness = 0.05      #mm 
        support_layer_thickness = 0.15           #mm
        repeated_membrane_thickness = 2 * membrane_thickness + feed_thickness + permeate_thickness + 2 * polysulfonic_layer_thickness + 2 * support_layer_thickness      #mm
        print('\nMembrane thickness:', '%s (mm)' %(repeated_membrane_thickness))

    elif module_selection == 'Custom':
        membrane_knowledge = input('''Do you know the thickness of the repeated unit of the filtration membrane? 
        < y > or < n >''')
        while membrane_knowledge not in possible_answers:
            print('''The module selection is not supported by this simulation.
            Select one of the options.''')  
            membrane_knowledge = input('''Do you know the thickness of the repeated unit of the filtration membrane? 
            < y > or < n >''')

        if membrane_knowledge == 'y':
            repeated_membrane_thickness = input('''What is the RO membrane thickness (mm)
            Default = 1.4117 ''') or 1.4117

        elif membrane_knowledge == 'n':   
            module_diameter = int(input('What is the module diameter? (mm)  '))
            permeate_tube_diameter = int(input('What is the module permeate tube diameter? (mm)  '))
            module_length = int(input('What is the module length? (m)  ')) 
            permeate_flow = int(input('What is the permeate flow rate? (m^3 / day)  ')) 
            max_feed_flow = int(input('What is the maximum feed flow rate? (m^3 / hour)  '))
            membrane_thickness = int(input('What is the filtration membrane thickness? (nm)  ')) / nm_over_mm
            feed_thickness = int(input('What is the feed spacer thickness? (mm) ')) 
            permeate_thickness = int(input('What is the permeate spacer thickness? (mm)  '))
            polysulfonic_layer_thickness = int(input('What is the polysulfonic layer thickness (mm)  '))
            support_layer_thickness = int(input('What is the support layer thickness? (mm)  ')) 
            repeated_membrane_thickness = 2 * membrane_thickness + feed_thickness + permeate_thickness + 2 * polysulfonic_layer_thickness + 2 * support_layer_thickness
            print('Thickness of the repeated membrane unit (mm): %s' %(repeated_membrane_thickness))
            
            
        
            
    #================================================================
    #parameterize and calculate simulation characteristics
    print('\nParameterize the simulation:')
    try:
        simulation_shifts = int(input('''Quantity of simulation shifts
        Default = 24  '''))
    except:
        simulation_shifts = 24
    try:
        cells_per_module = int(input('''Quantity of discretized module cells
        Default = 12  '''))
    except:
        cells_per_module = 12
    try:
        max_feed_flow = float(input('''Maximum feed flow rate (m^3 / hour)
        Default = 15.9  ''')) 
    except:
        max_feed_flow = 15.9
    try:
        quantity_of_modules = int(input('''Quantity of in-series RO modules 
        Default = 1  '''))
    except:
        quantity_of_modules = 1


    #calculate module properties
    module_cross_sectional_area = module_diameter**2 * math.pi / 4        #squared millimeters
    permeate_tube_cross_sectional_area = permeate_tube_diameter**2 * math.pi / 4     #squared millimeters
    filtering_layer_thicknes = (module_diameter - permeate_thickness) / 2         #millimeters
    filtration_cross_sectional_area = module_cross_sectional_area - permeate_tube_cross_sectional_area        #squared millimeters
    module_windings = filtering_layer_thicknes / repeated_membrane_thickness        #complete revolutions
    cell_length = module_length / cells_per_module     #meters

    #feed layer calculations
    feed_cross_sectional_area = (feed_thickness / repeated_membrane_thickness) * filtration_cross_sectional_area     #squared millimeters
    #print('\nFeed cross-sectional area:', feed_cross_sectional_area)
    feed_volume = feed_cross_sectional_area * module_length / mm_over_m**2   #cubic meters
    #print('Feed volume:', feed_volume)
    feed_mass = feed_volume * grams_over_liters_h2o * liters_over_cubic_meter / grams_over_kilograms    #kilograms, which assumes pure water for mass
    #print('Feed mass', feed_mass)
    feed_moles = feed_mass * grams_over_kilograms / grams_over_moles_h2o 
    #print('Feed moles:', feed_moles)

    #fluid flow calculations
    feed_velocity = max_feed_flow / (feed_cross_sectional_area / mm_over_m**2) / minutes_over_hour / seconds_over_minute     #meters / second
    reynolds_number = feed_velocity * module_length / kinematic_flow_velocity

    #module cell calculations
    feed_mass_cell = feed_mass / cells_per_module      #kg
    feed_moles_cell = feed_moles / cells_per_module    #moles

    #calculate simulation parameters that will populate the PHREEQC simulation 
    maximal_timestep = cell_length / feed_velocity * timesteps_over_cell         #seconds, from the Courant condition
    #print(maximal_timestep)
    permeate_removal_per_cell = maximal_timestep * permeate_flow / (seconds_over_day) * liters_over_cubic_meter * grams_over_liters_h2o / grams_over_moles_h2o / cells_per_module      #moles / cell
    #print(permeate_removal_per_cell)    



    #REACTION calculations 
    print('''First approach: linear change in moles
    Second approach: linear change in concentration''')
    permeate_approach = input('''What approach of permeate removal will you simulate? 
    < first > or < second >  ''')
    cfs = []
    cell_moles = []
    reaction_parameters = []
    iteration = 0
    previous_moles_removed = 0
    cumulative_cf = 1 
    for module in range(quantity_of_modules):
        if permeate_approach == 'first':
            if iteration == 0:
                initial_moles_cell = feed_moles_cell
            #print('\nInitial moles:\n', initial_moles_cell)
            print('''Permeate efficiency (PE):
            The PE embodies flux decreases through the module from fouling. 
            A value of 0 denotes complete fouling and the absence of permeate flux.
            A value of 1 denotes the absence of fouling and unabated permeate flux.''')
            permeate_efficiency = float(input('''What is the permeate efficiency (PE) for this module?
            0 < x < 1  '''))
            print('''Presesure gradient: 
            A value of -10 denotes a 10-log reduction in pressure at the end of the module realtive to the start of the module.
            A value of 0 denotes a homogeneous pressure throughout the module.''') 
            osmotic_pressure = float(input('''What is the pressure gradient through this module? 
            -10 < x < 0  '''))
            initial_moles_removed = permeate_removal_per_cell * 2 / (1 + math.exp(osmotic_pressure))
            final_moles_removed = initial_moles_removed * math.exp(osmotic_pressure)
            removed_moles_slope = ((final_moles_removed - initial_moles_removed) / (cells_per_module)) * permeate_efficiency
            average_moles_removed = (final_moles_removed + initial_moles_removed) / 2
            #print('Average moles removed: ', average_moles_removed)
            #print('\nInitial moles removed:\n', initial_moles_removed)
            #print('\nFinal moles removed:\n',final_moles_removed)
            #print('\nCells per module:\n',cells_per_module)
            print('Removed moles slope:', removed_moles_slope)

            permeate_efficiency_power = 1.1
            for cell in range(cells_per_module):
                removed_moles_in_cell = (cell * removed_moles_slope + initial_moles_removed) * permeate_efficiency**permeate_efficiency_power
                reaction_parameters.append(removed_moles_in_cell)
                previous_moles_removed += removed_moles_in_cell 
                #print(previous_moles_removed)
                
                                     

            #print('Removed moles: ', previous_moles_removed)
            cf = initial_moles_cell / (initial_moles_cell - previous_moles_removed)
            #print(cf)
            cumulative_cf *= cf
            #print('Cumulative CF:', cumulative_cf)

            initial_moles_cell -= previous_moles_removed
            previous_moles_removed = 0
            iteration += 1


        if permeate_approach == 'second':
            initial_cf = 1
            final_cf = float(input('''What is the final CF of your simulation? 
            1 < x '''))
            cf_slope = (final_cf - initial_cf) / cells_per_module
            #print(cf_slope)
            for cell in range(1,cells_per_module+1):
                cell_cf = cell * cf_slope + 1
                cfs.append(cell_cf)    

            #print(cfs)

            for cf in cfs:
                moles_to_be_removed = (feed_moles_cell / cf) - feed_moles_cell
                if iteration == 0:
                    reaction_parameters.append(moles_to_be_removed)
                if iteration > 0:
                    previous_moles_removed += reaction_parameters[iteration-1] 
                    #print(reaction_parameters)
                    #print('to_be:', moles_to_be_removed)
                    #print('previous:', previous_moles_removed)
                    reaction_parameter = moles_to_be_removed - previous_moles_removed
                    reaction_parameters.append(reaction_parameter)

                iteration += 1

            cf = cfs[-1]
            #print(cf)
            cumulative_cf *= cf
            #print('Cumulative CF: ', cumulative_cf)

    #print(reaction_parameters)   
    #print(len(reaction_parameters))

    #============================================================
    #the calculated reaction parameters will be added and printed to a generated PHREEQC input file
    reaction_cell_domain = range(1,cells_per_module+1)
    reaction_block = []
    reaction_block.append('\n')
    for cell in reaction_cell_domain:
        reaction_line = '''REACTION %s
            H2O -1; %s''' %(cell, reaction_parameters[cell-1])                
        #print(reaction_line)    
        reaction_block.append(reaction_line) 
        
        
    #============================================================
    #the TRANSPORT block is parameterized
    transport_block = []
    transport_line = '\nTRANSPORT'
    cells_line = '-cells\t\t\t%s' %(cells_per_module)
    shifts_line = '-shifts\t\t%s' %(simulation_shifts)
    lengths_line = '-lengths\t\t%s' %(cell_length)
    timestep_line = '-time_step\t\t%s' %(timesteps_over_cell)
    initial_time_line = '-initial_time\t\t0'    
    
    time_or_distance = input('''Would you like to examine scaling over distance in the module,
    or would you like to examine brine over time in the module?
    < Scaling > or < Brine >''')
    while time_or_distance != 'Scaling' and time_or_distance != 'Brine':
        print('''ERROR: The value is not one of the options.
        Select one of the choices to proceed.''' %(database_selection, mineral))  
        time_or_distance = input('''Would you like to examine scaling or effluent brine?
        < Scaling > or < Brine >''')
    if time_or_distance == 'Scaling':
        punch_cells_line = '-punch_cells\t\t1-%s' %(cells_per_module)
        punch_frequency_line = '-punch_frequency\t%s' %(cells_per_module)
    elif time_or_distance == 'Brine':
        punch_cells_line = '-punch_cells\t\t%s' %(cells_per_module)
        punch_frequency_line = '-punch_frequency\t1'       
    
    transport_block.extend((transport_line, cells_line, shifts_line, lengths_line, timestep_line, 
                           initial_time_line, punch_cells_line, punch_frequency_line))
    
    

def make_selected_output():
    '''
    Specify the output file after a PHREEQC simulation
    '''
    global selected_output_block
    
    first_line = '\nSELECTED_OUTPUT'
    
    file_name = input('''\nThe output file will now be parameterized.
    What is the name for your output file? A suffix of ", Scaling" or ", Brine" will be added to the end of the file name
    to facilitate future process of the data, and for explicit organization.
    Refrain from using decimals < . > or spaces in the name.''')
    file_name_line = '-file\t\t\t%s, %s.txt' %(file_name, time_or_distance)
    
    reaction_line = '-reaction\t\ttrue'
    temperature_line = '-temperature\t\ttrue'
    
    minerals_line = ''
    for mineral in minerals:
         minerals_line += ' %s' %(mineral)
    saturation_indices_line = '-saturation_indices\t' + minerals_line
    equilibrium_phases_line = '-equilibrium_phases\t' + minerals_line
    
    elements_line = ''
    for element in elements:
         elements_line += ' %s' %(element)
    total_elements_line = '-totals\t\t' + elements_line    
    
    ph_line = '-pH\t\t\ttrue'
    solution_line = '-solution'
    time_line = '-time\t\t\ttrue'
    distance_line = '-distance\t\ttrue'
    simulation_line = '-simulation\t\ttrue'
    high_precision_line = '-high_precision\ttrue'
    step_line = '-step'
    water_line = '-water'
    
    
    selected_output_block = []
    selected_output_block.extend((first_line,file_name_line,reaction_line,temperature_line,total_elements_line,
                                  saturation_indices_line,equilibrium_phases_line,ph_line,time_line,distance_line,
                                  simulation_line,high_precision_line,solution_line,step_line,water_line))
    
    
    
def export():
    """
    Export the data to a iPHREEQC input file
    """
    complete_lines = general_conditions + solution_block + equilibrium_phases_block + reaction_block + selected_output_block + transport_block 
    
    printing_block = input('''Would you like to view the input file?
    < y > or < n >\t''')
    if printing_block == 'y':
        print('\n\nInput file:\n')
        for element in complete_lines:
            print(element)
            
    file_number = 0
    if not os.path.exists('%s _PHREEQC input file_%s.pqi' %(datetime.date.today(), file_number)):
        input_file = open('%s _PHREEQC input file_%s.pqi' %(datetime.date.today(), file_number),'w')
        for line in complete_lines:
            input_file.write(line + '\n')
        input_file.close()
    elif os.path.exists('%s _PHREEQC input file_%s.pqi' %(datetime.date.today(), file_number)):
        while os.path.exists('%s _PHREEQC input file_%s.pqi' %(datetime.date.today(), file_number)):
            file_number += 1
        input_file = open('%s _PHREEQC input file_%s.pqi' %(datetime.date.today(), file_number),'w')
        for line in complete_lines:
            input_file.write(line + '\n')
        input_file.close()
            
    #return initial_conditions
    
    
make_general_conditions()
make_reactive_transport()
make_solutions()
make_equilibrium_phases()
make_selected_output()
export()

We will parameterize your path directory for the iPHREEQC software.
What is your computer drive directory?
    Default = C:
What is the program directory that stores iPHREEQC?
    Default = Program Files (x86)
What is the folder that contains iPHREEQC software?
    Default = USGS
What is the folder of iPHREEQC?
    Default = Phreeqc Interactive 3.6.2-15100
C:\Program Files (x86)\USGS\Phreeqc Interactive 3.6.2-15100
< pitzer >	
< phreeqc >	
< Amm >	
< ColdChem >	
< core10 >	
< frezchem >	
< iso >	
< llnl >	
< minteq >	
< minteq.v4 >	
< sit >	
< Tipping_Hurley >	
< wateq4f >	
What database do you select?pitzer
What is the title of your simulation?test
We will now parameterize the REACTION block.
< BW30-400 >
< Custom >
What RO module will you simuate?BW30-400

Membrane thickness: 1.4117000000000002 (mm)

Parameterize the simulation:
Quantity of simulation shifts
        Default = 24  
Quantity of discretized module cells
        Default = 12  
Maximum feed flow rate (m^3 / hour)
        